In [241]:
import numpy as np
import struct
import time
import socket

class Point:
    def __init__(self, x, y, r, g, b):
        self.x = int(x)
        self.y = int(y)
        self.r = int(r)
        self.g = int(g)
        self.b = int(b)

udp_ip = "192.168.1.141"
udp_port = 7200

IW_TYPE_0 = 0x00 # Turn off
IW_TYPE_1 = 0x01 # Period
IW_TYPE_2 = 0x02 # 16b X/Y + 8b R/G/B
IW_TYPE_3 = 0x03 # 16b X/Y + 16b R/G/B

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

def points_to_bytes(points):
    return b''.join([struct.pack('>B H H H H H', IW_TYPE_3, p.x, p.y, p.r, p.g, p.b) for p in points])

def set_scan_rate(rate):
    scan_period = int(1000000 / rate)  # [us] 10 - UINT32_MAX (4294967295)
    sock.sendto(struct.pack('>B I', IW_TYPE_1, scan_period), (udp_ip, udp_port))

def send_points(points, scan_rate, repeat=1):
    set_scan_rate(scan_rate)
    packet = b''.join([struct.pack('>B H H H H H', IW_TYPE_3, p.x, p.y, p.r, p.g, p.b) for p in points])
    for _ in range(repeat):
        sock.sendto(packet, (udp_ip, udp_port))
    # sock.sendto(struct.pack('>B', IW_TYPE_0), (udp_ip, udp_port))

def sweep_scan_rate(points, duration=20, min_rate=1, max_rate=10000):
    start_time = time.time()
    end_time = start_time + duration

    while time.time() < end_time:
        elapsed = time.time() - start_time
        scan_rate = int(min_rate + (max_rate - min_rate) * (elapsed / duration))
        scan_rate = min(scan_rate, max_rate)    
        send_points(points, scan_rate, repeat=1)
        
# ------------------------ Manual ------------------------
custom_anim = []
custom_anim.append(Point(0x0000, 0x0000, 0x0000, 0xFFFF, 0xFFFF))
custom_anim.append(Point(0xFFFF, 0x0000, 0xFFFF, 0x0000, 0xFFFF))
custom_anim.append(Point(0xFFFF, 0xFFFF, 0xFFFF, 0xFFFF, 0x0000))
custom_anim.append(Point(0x0000, 0xFFFF, 0xFFFF, 0xFFFF, 0xFFFF))
custom_anim.append(Point(0x0000, 0x0000, 0xFFFF, 0xFFFF, 0xFFFF))

# --------------------------------------------------------

# --------------------- Programmatic ---------------------
def generate_circle_points(num_points=100, radius=0x7FFF, color_cycle=True):
    points = []
    col_max = 0xFFFF
    for i in range(num_points):
        angle = 2 * np.pi * i / num_points
        x = int(0x8000 + radius * np.cos(angle))
        y = int(0x8000 + radius * np.sin(angle))

        if color_cycle:
            r = int((np.sin(angle) * 0.5 + 0.5) * col_max)
            g = int((np.sin(angle + 2*np.pi/3) * 0.5 + 0.5) * col_max)
            b = int((np.sin(angle + 4*np.pi/3) * 0.5 + 0.5) * col_max)
        else:
            r = g = b = col_max

        points.append(Point(x, y, r, g, b))
    return points

def generate_sinewave_points(num_points=50, amplitude=0x7FFF, color_cycle=True):
    points = []
    pos_max = 0xFFFF
    col_max = 0xFFFF
    for i in range(num_points):
        x = int(i / num_points * pos_max)
        y = int(0x8000 + amplitude * np.sin(2 * np.pi * i / num_points))
        
        if color_cycle:
            r = int((np.sin(2*np.pi * i / num_points) * 0.5 + 0.5) * col_max)
            g = int((np.sin(2*np.pi * i / num_points + 2*np.pi/3) * 0.5 + 0.5) * col_max)
            b = int((np.sin(2*np.pi * i / num_points + 4*np.pi/3) * 0.5 + 0.5) * col_max)
        else:
            r = g = b = col_max
        
        points.append(Point(x, y, r, g, b))
    points += list(reversed(points))
    return points
    
# --------------------------------------------------------

send_points(custom_anim, 500, 10)
# send_points(generate_circle_points(), 5000, 1000)
# send_points(generate_sinewave_points(), 5000, 1000)

# sweep_scan_rate(custom_anim, duration=5, min_rate=1, max_rate=500)
# sweep_scan_rate(generate_circle_points(), duration=10, min_rate=1, max_rate=2000)
# sweep_scan_rate(generate_sinewave_points(), duration=10, min_rate=1, max_rate=3000)